In [1]:
import pandas as pd
import os
import sys

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
cwd = os.getcwd()

education_folder_path = os.path.dirname(cwd)

sys.path.append(education_folder_path)

In [3]:
from global_functions import remove_spaces_in_columns, up_load_shp, up_load_df, drop_geo, change_cols_names, get_newest_date_file, get_output_folder_location

In [4]:
#בתי ספר וגנים מעיריית בית שמש
BShemesh=up_load_df(r'{}\background_files\betshemesh_muni\חינוך'.format(education_folder_path),'מצבת מוסדות חינוך תשפג משרדי')
BShemesh=remove_spaces_in_columns(BShemesh)

In [5]:
BShemesh=BShemesh[~BShemesh['הערות'].str.contains('גן', na=False)]

In [6]:
#קאורדינטות של מוסדות חינוך בכל הארץ ממשרד החינוך
moe_mosdot_coordinates_2022=up_load_df(r'{}\background_files\ministry_of_education\מוסדות'.format(education_folder_path),'moe_mosdot_coordinates_2022')
moe_mosdot_coordinates_2022=remove_spaces_in_columns(moe_mosdot_coordinates_2022)

In [7]:
moe_mosdot_coordinates_2022.rename(columns={'SEMEL_MOSAD': 'סמל_מוסד'}, inplace=True)
moe_mosdot_coordinates_2022.rename(columns={'SHEM_MOSAD': 'שם מוסד'}, inplace=True)
moe_mosdot_coordinates_2022.rename(columns={'ITM_X': 'coordinate_x'}, inplace=True)
moe_mosdot_coordinates_2022.rename(columns={'ITM_Y': 'coordinate_y'}, inplace=True)

moe_mosdot_coordinates_2022 = moe_mosdot_coordinates_2022.drop(columns=['UTM_X', 'UTM_Y', 'RAMAT_DIYUK_MIKUM'])

In [8]:
# מיזוג הטבלאות לפי העמודה "סמל_מוסד"
BShemesh = pd.merge(BShemesh, moe_mosdot_coordinates_2022[['סמל_מוסד', 'coordinate_x', 'coordinate_y']],
                     on='סמל_מוסד', how='left')

In [ ]:
BShemesh=BShemesh.drop(columns=['שכונה', 'כתובת', 'טלפון', 'פקס', 'מייל', 'שם_מנהל/ת',
                                    'כתובת_מנהל', 'נייד', 'שם_מזכיר/ה', 'נייד.1', 'הערות', 'מפקח', 'קב"ט_מוס"ח',
                                    'גן_טרום_חובה', 'גן_חובה', 'גני_חינוך_מיוחד', 'סה"כ_גנים'])
BShemesh.head()

In [ ]:
# רשימת שמות השכבות (א' עד ו')
grades = ['א', 'ב', 'ג', 'ד', 'ה', 'ו', 'ז', 'ח', 'ט', 'י', 'יא', 'יב']

# לולאה שמבצעת את הסכימה והמחיקה עבור כל שכבה
for i, grade in enumerate(grades):
    # איתור העמודות הרלוונטיות לכל שכבה (כולל כיתה מקדמת אם קיימת לאותה שכבה)
    grade_columns = [col for col in BShemesh.columns if col.startswith(f'כיתה_{grade}')]
    
    # הוספת העמודה של הכיתה המקדמת של אותה שכבה אם קיימת
    if f'כיתה_מקדמת.{i}' in BShemesh.columns:
        grade_columns.append(f'כיתה_מקדמת.{i}')
    elif 'כיתה_מקדמת' in BShemesh.columns and i == 0:
        grade_columns.append('כיתה_מקדמת')
    
    # סכימת העמודות אם יש כיתות רלוונטיות לשכבה הזו
    if grade_columns:
        BShemesh[f'כיתה_{grade}'] = BShemesh[grade_columns].sum(axis=1, skipna=True)
        
        # מחיקת העמודות שסוכמו
        BShemesh.drop(columns=grade_columns, inplace=True)

# institutions_table = pd.merge(institutions_table, BShemesh, on='סמל_מוסד', how='outer')
# institutions_table = institutions_table.dropna(axis=1, how='all')

BShemesh.head()

In [11]:
# יצירת עמודה "סה"כ" שמכילה את סכום כל העמודות הרלוונטיות
BShemesh['סה"כ_תלמידים'] = BShemesh[['סה"כ_יסודי_מ"מ_וממ"ד', 'סה"כ_יסודי_חרדי', '_סה"כ_על_יסודי_מ"מ_וממ"ד', 'סה"כ_על_יסודי_חרדי']].sum(axis=1)

# הסרת העמודות הישנות אם אינך זקוק להן יותר
BShemesh=BShemesh.drop(columns=['סה"כ_יסודי_מ"מ_וממ"ד', 'סה"כ_יסודי_חרדי', '_סה"כ_על_יסודי_מ"מ_וממ"ד', 'סה"כ_על_יסודי_חרדי'])

In [12]:
#sector
BShemesh.loc[BShemesh['מגזר']=='חרדי','sector'] = 2
BShemesh.loc[BShemesh['מגזר']=='ממ"ח','sector'] = 2
BShemesh.loc[BShemesh['מגזר']=='מ"מ','sector'] = 3
BShemesh.loc[BShemesh['מגזר']=='ממ"ד','sector'] = 3
BShemesh.loc[BShemesh['מגזר']=='עירוני','sector'] = 3

In [13]:
# סינון שורות שבהן העמודה "סה"כ" אינה שווה לאפס
BShemesh=BShemesh[BShemesh['סה"כ_תלמידים'] != 0]

In [14]:
# חלוקה לשכבות גיל
ages = {
    'ele': ['כיתה_א', 'כיתה_ב', 'כיתה_ג', 'כיתה_ד', 'כיתה_ה', 'כיתה_ו'],
    'mid': ['כיתה_ז', 'כיתה_ח', 'כיתה_ט'],
    'high': ['כיתה_י', 'כיתה_יא', 'כיתה_יב']
}

# חישוב מספר התלמידים לכל שכבה
BShemesh['ele_stu'] = BShemesh[ages['ele']].sum(axis=1)
BShemesh['mid_stu'] = BShemesh[ages['mid']].sum(axis=1)
BShemesh['high_stu'] = BShemesh[ages['high']].sum(axis=1)

BShemesh_final = BShemesh.drop(['כיתה_א', 'כיתה_ב', 'כיתה_ג', 'כיתה_ד', 'כיתה_ה', 'כיתה_ו', 'כיתה_ז', 'כיתה_ח', 'כיתה_ט', 'כיתה_י', 'כיתה_יא', 'כיתה_יב'], axis=1)

In [ ]:
BShemesh.head()

In [ ]:
BShemesh_final.head()

In [ ]:
is_unique = BShemesh_final['סמל_מוסד'].is_unique
is_unique

In [ ]:
# מציאת שורות עם ערכים כפולים בעמודה סמל_מוסד
duplicated_rows = BShemesh_final[BShemesh_final.duplicated(subset=['סמל_מוסד'], keep=False)]
duplicated_rows

In [ ]:
BShemesh_final.to_excel('BShemesh_final.xlsx', index=False)